In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Layer, Input, Flatten, Dense, Embedding, Dropout, Multiply, Concatenate
# from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean, MeanSquaredError, MeanAbsoluteError

# Data Generate

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [4]:
df = getDF('Industrial_and_Scientific_5.json.gz')

In [5]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"11 27, 2017",A1JB7HFWHRYHT7,B0000223SI,{'Size:': ' 1-(Pack)'},Alex W.,This worked really well for what I used it for...,Couldn't have been happier with it's performance,1511740800,NaN,NaN
1,5.0,True,"11 4, 2017",A2FCLJG5GV8SD6,B0000223SI,{'Size:': ' 1-(Pack)'},Randall Harris,Fast cutting and good adheasive.,Good paper.,1509753600,NaN,NaN
2,5.0,False,"10 27, 2017",A3IT9B33NWYQSL,B0000223SI,{'Size:': ' 1-(Pack)'},A. C.,Worked great for my lapping bench. I would li...,Handy!,1509062400,NaN,NaN
3,4.0,True,"01 13, 2018",AUL5LCV4TT73P,B0000223SK,{'Size:': ' 1-Pack'},TnT,As advertised,As advertised,1515801600,NaN,NaN
4,5.0,True,"10 7, 2017",A1V3I3L5JKO7TM,B0000223SK,{'Size:': ' 1-Pack'},John Jones,seems like a pretty good value as opposed to b...,seems like a pretty good value as opposed to b...,1507334400,NaN,NaN


In [6]:
df = df[["reviewerID", "asin", "overall"]]

In [7]:
le = LabelEncoder()
df["reviewerID"] = le.fit_transform(df["reviewerID"].values)
df["asin"] = le.fit_transform(df["asin"].values)

In [8]:
df.head()

,reviewerID,asin,overall
0,1557,0,5.0
1,4282,0,5.0
2,7415,0,5.0
3,10602,1,4.0
4,2574,1,5.0


In [9]:
user_num = df["reviewerID"].max() + 1
item_num = df["asin"].max() + 1
print(f"user_num: {user_num}")
print(f"item_num: {item_num}")

user_num: 11041
item_num: 5334


In [10]:
def preprocessing_data(df): # Split data into 8/2
    TRAIN_SIZE = 0.8
    VALIDADTION_SIZE = 0.5
    ratings = shuffle(df, random_state=1)
    cutoff = int(TRAIN_SIZE * len(ratings))
    train_df = ratings.iloc[:cutoff]
    validation_test_df = ratings.iloc[cutoff:]
    validation_cutoff = int(VALIDADTION_SIZE * len(validation_test_df))
    validation_df = validation_test_df.iloc[:validation_cutoff]
    test_df = validation_test_df.iloc[validation_cutoff:]

    return train_df, validation_df, test_df

In [11]:
train_df, validation_df, test_df = preprocessing_data(df)

In [12]:
print(f"train_df.shape: {train_df.shape}")
print(f"validation.shape: {validation_df.shape}")
print(f"test_df.shape: {test_df.shape}")

train_df.shape: (61656, 3)
validation.shape: (7707, 3)
test_df.shape: (7708, 3)


In [13]:
train_dict = {"User": train_df["reviewerID"].values.astype(np.float32),
              "Item": train_df["asin"].values.astype(np.float32),
              "Rating": train_df["overall"].values.astype(np.float32)}

validation_dict = {"User": validation_df["reviewerID"].values.astype(np.float32),
              "Item": validation_df["asin"].values.astype(np.float32),
              "Rating": validation_df["overall"].values.astype(np.float32)}

test_dict = {"User": test_df["reviewerID"].values.astype(np.float32),
              "Item": test_df["asin"].values.astype(np.float32),
              "Rating": test_df["overall"].values.astype(np.float32)}

In [14]:
print(train_dict["User"])
print(train_dict["Item"])
print(train_dict["Rating"])

[ 6572. 10810. 10883. ...  4520.  2452.  5626.]
[5262.  871.  842. ... 1069. 4059. 2690.]
[5. 5. 5. ... 4. 5. 5.]


In [15]:
print(validation_dict["User"])
print(validation_dict["Item"])
print(validation_dict["Rating"])

[7273.  779. 6659. ... 6855. 6185.  551.]
[4186. 4333.  623. ... 3324. 5269. 1056.]
[5. 5. 5. ... 5. 5. 5.]


In [16]:
print(test_dict["User"])
print(test_dict["Item"])
print(test_dict["Rating"])

[7793.  229. 2730. ... 4466.   22. 6359.]
[ 477. 4659. 5289. ... 3109. 3659.  330.]
[5. 5. 5. ... 5. 5. 5.]


In [22]:
train_ds = tf.data.Dataset.from_tensor_slices((train_dict["User"], train_dict["Item"], train_dict["Rating"]))
train_ds = train_ds.shuffle(1000).batch(1024)

validation_ds = tf.data.Dataset.from_tensor_slices((validation_dict["User"], validation_dict["Item"], validation_dict["Rating"]))
validation_ds = validation_ds.shuffle(1000).batch(512)

test_ds = tf.data.Dataset.from_tensor_slices((test_dict["User"], test_dict["Item"], test_dict["Rating"]))
test_ds = test_ds.shuffle(1000).batch(512)

# Utils

In [23]:
class MLP(Layer):
    def __init__(self, first_node, n_layer):
        super(MLP, self).__init__()
        n_node = first_node
        self.mlp_layer = []
        for i in range(n_layer):
            self.mlp_layer.append(Dense(units = n_node, activation = "relu"))
            self.mlp_layer.append(Dropout(0.1))
            n_node //= 2

    def call(self, input):
        x = input
        for layer in self.mlp_layer:
            x = layer(x)
        return x

In [24]:
class NeuralCF(Model):
    def __init__(self, N, M, K):
        super(NeuralCF, self).__init__()
        self.user_embedding = Embedding(N, K)
        self.item_embedding = Embedding(M, K)
        self.user_flatten = Flatten()
        self.item_flatten = Flatten()
        self.GMF = Multiply()
        self.first_Concat = Concatenate()
        self.MLP = MLP(32, 3)
        self.second_Concat = Concatenate()
        self.prediction_layer = MLP(32, 3)
        self.output_layer = Dense(1,activation = "linear")

    def call(self, inputs):
        # User, Item Embedding
        user_emb = self.user_embedding(inputs[0])
        item_emb = self.item_embedding(inputs[1])
        user_emb = self.user_flatten(user_emb)
        item_emb = self.item_flatten(item_emb)

        # GMF Layer
        gmf_layer = self.GMF([user_emb, item_emb])

        # MLP Layer
        MLP_layer_input = self.first_Concat([user_emb, item_emb])
        MLP_layer = self.MLP(MLP_layer_input)

        # Prediction Layer
        NeuMF_input = self.second_Concat([gmf_layer, MLP_layer])
        NeuMF_layer = self.prediction_layer(NeuMF_input)
        output = self.output_layer(NeuMF_layer)
        return output

In [25]:
def load_metrics():
    global train_loss, train_acc
    global validation_loss, validation_acc
    global test_loss, test_rmse, test_mae

    train_loss = Mean()
    validation_loss = Mean()
    test_loss = Mean()

    test_rmse = MeanSquaredError()
    test_mae = MeanAbsoluteError()

In [26]:
@tf.function
def trainer():
    global train_ds, train_loss, model
    global optimizer, loss_object
    # cardinality = np.sum([1 for i in train_ds])

    for x, y, z in train_ds:
        with tf.GradientTape() as tape:
            predictions = model([x, y])
            # print(f"predictions: {predictions}")
            loss = loss_object(z, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)

@tf.function
def validation():
    global validation_ds, model, loss_object
    global validation_loss, validation_acc
    # cardinality = np.sum([1 for i in validation_ds])
    for x, y, z in validation_ds:
        predictions = model([x, y])
        loss = loss_object(z, predictions)

        validation_loss(loss)

@tf.function
def tester():
    global test_ds, model, loss_object
    global test_loss, test_rmse, test_mae
    # cardinality = np.sum([1 for i in test_ds])
    for x, y, z in test_ds:
        predictions = model([x, y])
        loss = loss_object(z, predictions)

        test_loss(loss)
        test_rmse(y, predictions)
        test_mae(y, predictions)

# Initiation

In [27]:
EPOCHS = 10
LR = 0.001

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = Adam(learning_rate = LR)

load_metrics()

# Model

In [28]:
model = NeuralCF(user_num, item_num, 32)

In [29]:
for epoch in tqdm(range(EPOCHS)):
    trainer()
    validation()
    print(f'\n train_loss: {(train_loss.result()):.3f}, validation_loss: {(validation_loss.result()):.3f}')

    train_loss.reset_states()
    validation_loss.reset_states()

 10%|█         | 1/10 [00:16<02:31, 16.78s/it]


 train_loss: 14.367, validation_loss: 1.786


 20%|██        | 2/10 [00:22<01:23, 10.39s/it]


 train_loss: 1.081, validation_loss: 0.836


 30%|███       | 3/10 [00:24<00:45,  6.55s/it]

...


 train_loss: 0.470, validation_loss: 0.780


 90%|█████████ | 9/10 [00:35<00:02,  2.10s/it]


 train_loss: 0.429, validation_loss: 0.827


100%|██████████| 10/10 [00:36<00:00,  3.62s/it]


 train_loss: 0.381, validation_loss: 0.837


In [31]:
tester()
print(f'Test_loss: {(test_loss.result()):.3f}')

Test_loss: 0.826
